# Mini-projet n°3 - ViT vs CNN

L'objectif de ce mini-projet est de comparer les performances de deux architectures différentes pour de la classification d'image : les Vision Transformers (ViT) et les Convolutional Neural Networks (CNN).

## Explication de l'architecture Transformers

Les transformers ont initialement étaient inventés pour des problèmes NLP mais on peut aussi l'utiliser -comme ici- pour de la classification d'images. 

Avant le block Transformer, il y a l'input embedding qui se traduit par de la vectorisation des patchs d'images (découpe de l'image en patchs). Pour que le modèle sache où est ce que le patch se situe sur l'image et donc garder l'information spatiale, on ajoute un vecteur de position créé par le positional encoding.
Après avoir fait cela, on entre dans l'encodeur qui est tout d'abord défini par un multi head attention. Cette "opération"/ ce procédé sert à capturer les diverses relations dans les données. C'est une transformation sur les données d'entrées qui permettent de prendre en compte les pondérations et les différences entre les éléments. **A developper avec key, query etc**